This is an IPython notebook that you can run/edit interactively. If you google for ipython notebook tutorial you'll find more information. In this class, Google is your friend, and I encourage you to use it extensively. Here we will download a genome and use it to illustrate the principles of the binomial distribution we went over in class. Note that you can edit the cells and redo the calculations, e.g. to try different parameters or to do it on a different genome. Awesome, right? more text here

# Part 0: learning the IPython notebook

The IPython notebook creates an interpreter live on the web for you. Basically the way it works is: you type in the code, and it executes it. For example, in the block below, we will calculate 2+2:

In [1]:
2 + 3

5

If you don't see a line that says "Out", click the play button (the triangle facing forward) at the top of the window, just under the menu. You'll then see a line that says Out[x]: 4  (where x is some number that depends on how long you have been playing with this session) containing the result. If you want to calculate something else, e.g. 2+3, just click in the cell, update the number, and click play again. If you see a *, that means the result is calculating. Sometimes the session drops because the web isn't entirely reliable; in that case, just reload the notebook.

If you create a variable, it persists from cell to cell. For example:

In [2]:
x = 4

In the cell above, we defined x as 3. x sticks around to the cell below:

In [3]:
print x*2

8


If you change x in the first cell, then from the Cell menu choose Run All, both cells will update. So you can update a whole calculation by changing an earlier cell. You also have access to everything in matplotlib, e.g. its interative graphics. For example:

In [48]:
from numpy.random import binomial
import matplotlib.pyplot as plt

plt.hist(binomial(100,0.5,1000000))

(array([  9.80000000e+01,   3.19000000e+03,   2.48630000e+04,
          1.55277000e+05,   2.76712000e+05,   3.55935000e+05,
          1.55525000e+05,   2.50870000e+04,   3.20300000e+03,
          1.10000000e+02]),
 array([ 27. ,  31.6,  36.2,  40.8,  45.4,  50. ,  54.6,  59.2,  63.8,
         68.4,  73. ]),
 <a list of 10 Patch objects>)

If you update the parameters of the binomial, you'll get a new graph, and as you're typing it even tells you what parameters the function takes. What's above is the example we did in class (binomial with 100 trials, probability of success 0.2, and then trying that 1000 times to see what the distribution of successes looks like). Try modifying p to 0.5 and see what happens.

# Part 1: Getting a genome

The first step is to grab a genome...we will do this by using urllib2, which lets us grab a resource over the internet. I was going to do this with IMG, but remember that IMG has that "data use agreement" that you have to click on, which we can't do with an automated system. So we will get the genomes from NCBI instead. This reinforces the utility of free, public data without individual license agreements.

First, let's find out what bacteria there are:

In [41]:
from urllib2 import urlopen
ncbi = urlopen('ftp://ftp.ncbi.nlm.nih.gov/genomes/Bacteria')
print ncbi.read()

URLError: <urlopen error ftp error: [Errno ftp error] 550 Bacteria: No such file or directory>

I'm just going to pick the directory that happens to be at the top of the list, but you can paste in any of the directories below if you want to look at another genome. It's as simple as that -- awesome, right?

In [49]:
genome_dir = urlopen('ftp://ftp.ncbi.nlm.nih.gov/genomes/Bacteria/Acaryochloris_marina_MBIC11017_uid58167') #note that I don't need to redo the import
print genome_dir.read()

URLError: <urlopen error ftp error: 550 genomes/Bacteria: No such file or directory>

So what's going on here? The family of files for each accession give the record in different formats. Remember that fna is the nucleotide, faa is the fasta for amino acids. gff is a feature table, gbk is a genbank file, etc. Why are there a bunch of accessions? Probably because this organism has a lot of plasmids. Let's confirm that by reading one of the smaller files in .gbk format, which as the richest annotations:

In [50]:
print urlopen('ftp://ftp.ncbi.nlm.nih.gov/genomes/Bacteria/Acaryochloris_marina_MBIC11017_uid58167/NC_009934.gbk').read()

URLError: <urlopen error ftp error: 550 genomes/Bacteria/Acaryochloris_marina_MBIC11017_uid58167: No such file or directory>

...yep, it's a plasmid. Now the whole genome is a bit bigger, so we might just want to look at the first part of the genbank file for that to confirm.

In [51]:
#print urlopen('ftp://ftp.ncbi.nlm.nih.gov/genomes/Bacteria/Acaryochloris_marina_MBIC11017_uid58167/NC_009925.gbk').read()[:1000]

Confirmed! (this is cut off because we're just looking at the first 1000 characters) -- note that it says that it's the complete genome in the DEFINITION line. Writing a parser for the GenBank flat file format is not very fun (many more evil textbooks leave it as an exercise after showing you how to write a FASTA parser), so we'll focus on the FASTA files for now. Reading a single bacterial genome into memory isn't a big deal (this is about 6.5 MB, the size of about a 10-min MP3), but you might want to be a bit more careful for eukaryotes, which are a lot bigger.

In [52]:
#genome = urlopen('ftp://ftp.ncbi.nlm.nih.gov/genomes/Bacteria/Acaryochloris_marina_MBIC11017_uid58167/NC_009925.fna').read()
#print genome

Note that this gives us the whole genome as one fasta file. If we want the genes instead, we can get them by:

In [53]:
gene_url = urlopen('ftp://ftp.ncbi.nlm.nih.gov/genomes/Bacteria/Acaryochloris_marina_MBIC11017_uid58167/NC_009925.ffn')

URLError: <urlopen error ftp error: 550 genomes/Bacteria/Acaryochloris_marina_MBIC11017_uid58167: No such file or directory>

Notice how they essentially all start with ATG, as expected. Regrettably this just gives you the sequences, not any sort of annotation that would tell you what the sequence actually is (they do show you the start and end point on the chromosome). But tackling the annotation issue is a task for another time.

In [54]:
genes = gene_url.read()

NameError: name 'gene_url' is not defined

In [55]:
print genes

NameError: name 'genes' is not defined

# Part 2: Reading the gene/genome data

OK, now we have the genome and the genes. But what do we do next? Well, remember the idea is to read the gene data into a string (if you don't know what a "string" is, google "python string" and click on the second result, for python 2.7). We started on a fasta parser in class, but didn't quite finish it off. So let's finish it off now. We could test this on the whole genome, but it would be really annoying to check every record to make sure it actually did it. So let's do a much simpler fasta record. We are going to set this up as a multi-line string:

In [56]:
simplest_fasta = """>abc
ATGC"""

In [57]:
multiple_fasta = """>abc
ATGC
CCCC
>def
AAAA
>qaz
ATGC
CCCC
"""

Remember that we want to be able to read both a single fasta record from a file, and also multiple fasta records. Let's set up the exact FASTA parser as it was at the end of class on Thursday:

In [58]:
def read_fasta(infile): #this is where we left it at the end of class
    curr_seq = []
    for line in infile:
        line = line.strip()
        if line.startswith('>'): #label line
            if curr_seq:
                yield curr_seq[0], ''.join(curr_seq[1:])
            curr_seq = [line]
        else:
            curr_seq.append(line)
    if curr_seq:
        yield curr_seq[0], ''.join(curr_seq[1:])

...and let's see the results:

In [59]:
print list(read_fasta(simplest_fasta))

[('>', 'abcATGC')]


oops, that's not right -- it's just reading a single >, and running the label and sequence together. What's the problem? Remember that the fasta parser is supposed to work on lines from a file, not on strings, so we need to use the StringIO object to turn the string into something that looks like a file.

In [60]:
from StringIO import StringIO
print list(read_fasta(StringIO(simplest_fasta)))

[('>abc', 'ATGC')]


That's more like it! But what happens when we use the more complex example?

In [61]:
print list(read_fasta(StringIO(multiple_fasta)))

[('>abc', 'ATGCCCCC'), ('>def', 'AAAA'), ('>qaz', 'ATGCCCCC')]


...looking good!

# Part 3: Calculating the GC content

The gc content code is a lot simpler:

In [62]:
2/float(3)

0.6666666666666666

In [69]:
def calc_gc(s):
    gc = 0.0
    count = 0.0
    for i in s:
        if i == 'G' or i == 'C':
            gc += 1
        count += 1
    return gc/count

So let's string them together. Remember that read_fasta returns pairs of (label, sequence) tuples -- if you don't know what a tuple is, see here: http://lmgtfy.com/?q=tuple+python -- so we need to extract the sequence and run calc_gc against each sequence. We can write that like this: 

In [70]:
result = []
for label, seq in read_fasta(StringIO(multiple_fasta)):
    result.append(calc_gc(seq))
print result

[0.75, 0.0, 0.75]


...or more compactly with the map function and a list comprehension:

In [71]:
print map(calc_gc, [i[1] for i in read_fasta(StringIO(multiple_fasta))])

[0.75, 0.0, 0.75]


In general, there are lots of different choices in how you write a given piece of code. Readability as well as efficiency can be important. So let's do exactly the same thing on all the genes in the genome we read in earlier, and plot a histogram:

In [72]:
hist(map(calc_gc, [i[1] for i in read_fasta(StringIO(genes))]), bins=30)

That looks a bit skewed. Remember that the binomial assumes that you have a fixed number of trials, and genes vary in lengths. Let's look at that length distribution:

In [73]:
hist(map(len, [i[1] for i in read_fasta(StringIO(genes))]), bins=30)

In [74]:
for label, seq in read_fasta(StringIO(genes)):
    if len(seq) > 1000:
        print label

NameError: name 'genes' is not defined

OK, so there aren't all that many genes total, and there are some very short and very long ones. This could point to a problem with annotation in this particular genome. Why don't we try clipping out the first 50 bases (disregarding the first three bases which are almost always ATG), throw away the short genes, and replot?

In [75]:
hist(map(calc_gc, [i[1][3:103] for i in read_fasta(StringIO(genes)) if len(i[1])>103]), bins=30)

...that's looking a lot better! The code's getting a bit ugly, though; let's do the exact same thing but make it a bit easier to read by unpacking the tuple:

In [76]:
hist(map(calc_gc, [seq[3:53] for label, seq in read_fasta(StringIO(genes)) if len(seq)>53]), bins=30)

OK, now let's calculate the average GC content so we can fit the binomial:

In [77]:
mean_gc = average(map(calc_gc, [seq[3:53] for label, seq in read_fasta(StringIO(genes)) if len(seq)>53]))
print mean_gc

NameError: name 'genes' is not defined

In [78]:
num_genes = len(list(read_fasta(StringIO(genes))))
print num_genes

NameError: name 'genes' is not defined

Only 132 genes? hmmm...maybe good to rerun with a different genome (you can do this by just changing the lines of code above to point to a different genome and its accession, left as an exercise to try out). Note that I am very inefficiently re-parsing the FASTA file each time -- on the one hand it would make more sense to just read in the result and save it, but on the other hand today's computers are fast so it's a trivial cost, and faster than e.g. hitting NCBI again to download the genome when you reload the notebook. So anyway, let's plot the binomial with the same parameters:

In [79]:
hist(binomial(50, mean_gc, num_genes), bins=30)

Of course, this is counting the number of successes, whereas what we want is the GC content (i.e. the probability of success) in each window, so we actually need to divide the results by the length of the gene:

In [80]:
hist(binomial(50, mean_gc, num_genes)/50.0, bins=30)

...which looks pretty similar. But looks aren't really enough: let's compare to the expected binomial distribution. To do this, we need to set up those functions for calculating the binomial from class:

In [81]:
def factorial(x):
    """calculate factorial of x"""
    if x < 0:
        raise ValueError, "Factorial only works on non-negative numbers"
    result = 1
    for i in range(1,x+1):
        result *= i
    return result

def binomial_coeff(x, k):
    """return binomial coefficient given x and k"""
    return factorial(x)/(factorial(k)*factorial(x-k))

def binomial_pmf(x,k,p):
    """return number of successes given x, k and p"""
    return binomial_coeff(x,k) * p**k * (1-p)**(x-k)

Because we now want to do multiple plots on the same axes, we need to do a little more work now rather than just using the most basic built-in functionality. So let's do that:

In [86]:
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots(1)
simulated_data = binomial(50, mean_gc, num_genes)
ax1.hist(simulated_data, bins=arange(0,51),alpha=0.5) #alpha sets transparency so we can see several series at once
real_data = map(calc_gc, [seq[3:53] for label, seq in read_fasta(StringIO(genes)) if len(seq)>53])
#note that real_data is fractions so need to convert back to counts
ax1.hist([i*50 for i in real_data], bins=arange(0,51), alpha=0.5,fc='red') #fc sets the face color
exact_distribution = [num_genes*binomial_pmf(50,i,mean_gc) for i in range(51)]
ax1.plot(exact_distribution, lw=3, color='green')

NameError: name 'mean_gc' is not defined

...so this looks pretty good visually. Testing the fit statistically will be a topic for a later class/demo. Remember that you can try this out on other genomes just by updating the numbers; some of the stuff like the length window of 50 bases is hard-coded and requires updates in multiple places to change, though. This demonstrates why it is nice to use variables rather than scattering hardcoded numbers around. We can also do multinomial sampling:

In [87]:
from numpy.random import multinomial
result = multinomial(100, [0.2,0.4,0.25,0.35]) #the array has the freqs of the different items -- you can think of positions 0, 1, 2, 3 as ATGC
print result

[23 35 26 16]


...or do draws for frequencies for a whole lot of seqeunces at once:

In [88]:
result = multinomial(100, [0.2,0.4,0.25,0.35], 20) #the array has the freqs of the different items -- you can think of positions 0, 1, 2, 3 as ATGC
print result #now has base frequencies for 20 100-base seqs

[[24 34 24 18]
 [15 43 26 16]
 [12 46 31 11]
 [22 42 21 15]
 [28 36 23 13]
 [21 39 22 18]
 [27 34 22 17]
 [ 9 47 28 16]
 [16 40 28 16]
 [18 39 28 15]
 [25 38 27 10]
 [22 40 26 12]
 [15 47 23 15]
 [20 44 19 17]
 [16 48 22 14]
 [23 39 23 15]
 [17 36 36 11]
 [15 41 31 13]
 [21 45 25  9]
 [19 37 30 14]]


...and the calc_gc code can be easily modified to count all four bases as well:

In [89]:
def base_freqs(s):
    result = zeros(4,dtype=int)
    for i in s:
        if i == 'A':
            result[0] += 1
        elif i == 'T':
            result[1] += 1
        elif i == 'G':
            result[2] += 1
        elif i == 'C':
            result[3] += 1
    return result

In [90]:
print base_freqs('AATC')

[2 1 0 1]


We can then use the multinomial distribution to test whether the frequencies of all four bases are what we expect, although again we will elaborate on this example later in the course.

# Part 4: How good is the approximation between the binomial and the normal?

Remember that we can approximate the binomial using $$\mu=np$$ and $$\sigma=\sqrt{(np(1-p))}$$ Let's see how good this approximation is. You can change the parameters below to update the plot.

In [92]:
import matplotlib.pyplot as plt

n = 10
p = 0.1
exact_probabilities = [binomial_pmf(n,i,p) for i in range(n+1)]
print 'Exact binomial probabilities:', exact_probabilities
fig, ax1 = plt.subplots(1)
ax1.plot(exact_probabilities,color='black')
mu = n*p
sigma = sqrt(n*p*(1-p))
print "MU", mu, 'SIGMA', sigma
x_range = arange(0,n+1,0.1)
normal_approx = normpdf(x_range, mu, sigma)
ax1.plot(x_range, normal_approx, color='red')


Exact binomial probabilities: [0.3486784401000001, 0.3874204890000001, 0.19371024450000007, 0.05739562800000002, 0.011160261000000003, 0.0014880348000000005, 0.00013778100000000007, 8.748000000000003e-06, 3.6450000000000023e-07, 9.000000000000004e-09, 1.0000000000000006e-10]
MU 1.0 SIGMA 0.948683298051


NameError: name 'normpdf' is not defined

Try modifying the values of n and p in the code block above and hit play again to get an idea of how well the approximation works under different conditions.